In [30]:
import numpy as np
import pandas as pd
import os
pd.options.display.max_rows = 10
pd.options.display.width = 200
np.set_printoptions(precision = 4, suppress = True)

data = pd.read_csv("C:/Users/Armin/Documents/VU/thesis/original/02_MyMovez/5. Fitbit data/Fitbit_W17_cleaned_imputation.csv", sep= ';') 
print(data)

School  Class  Child  Sex   Age  Wave  Day       Date  Participated    Steps  Minutes_MVPA  Imputed
0          22     52    902    1  13.0     1    2  29-1-2016           1.0  15921.0         124.0  imputed
1          22     52    902    1  13.0     1    1  28-1-2016           1.0   3426.0           0.0  imputed
2          22     52    902    1  13.0     1    3  30-1-2016           1.0  12311.0         237.0  imputed
3          22     52    902    1  13.0     2    3   2-4-2016           1.0  23583.0          17.0  imputed
4          22     52    902    1  13.0     2    2   1-4-2016           1.0   1000.0           0.0  imputed
...       ...    ...    ...  ...   ...   ...  ...        ...           ...      ...           ...      ...
26461      59    307   6131    0  16.0     7    3  20-5-2018           1.0  17586.0          34.0  imputed
26462      59    307   6131    0  16.0     7    2  19-5-2018           1.0   8891.0          30.0  imputed
26463      59    307   6133    0  15.0     7